# Cholesky Decomposition

<img src="http://www.cerebralmastication.com/wp-content/uploads/2010/09/39-cholesky-250x300.jpg" width="200" align="center"/>

* A basic tenet in numerical analysis: 

> The structure should be exploited whenever solving a problem.

Common structures include: symmetry, definiteness, sparsity, Kronecker product, low rank, ...

* LU decomposition (Gaussian Elimination) is **not** used in statistics so often because most of time statisticians deal with positive (semi)definite matrix.

* Consider solving the normal equation 
$$
    \mathbf{X}^T \mathbf{X} \beta = \mathbf{X}^T \mathbf{y}
$$
for linear regression. The coefficient matrix $\mathbf{X}^T \mathbf{X}$ is symmetric and positive semidefinite. How to exploit this structure?

## Cholesky decomposition

* **Theorem**: Let $\mathbf{A} \in \mathbb{R}^{n \times n}$ be symmetric and positive definite. Then $\mathbf{A} = \mathbf{L} \mathbf{L}^T$, where $\mathbf{L}$ is lower triangular with positive diagonal entries and is unique.  
**Proof** (by induction):  
If $n=1$, then $\ell = \sqrt{a}$. For $n>1$, the block equation
\begin{eqnarray*}
\begin{pmatrix}
a_{11} & \mathbf{a}^T \\ \mathbf{a} & \mathbf{A}_{22}
\end{pmatrix} =
\begin{pmatrix}
	\ell_{11} & \mathbf{0}_{n-1}^T \\ \mathbf{l} & \mathbf{L}_{22}
\end{pmatrix}
\begin{pmatrix}
	\ell_{11} & \mathbf{l}^T \\ \mathbf{0}_{n-1} & \mathbf{L}_{22}^T
\end{pmatrix}.
\end{eqnarray*}
has solution
\begin{eqnarray*}
	\ell_{11} &=& \sqrt{a_{11}} \\
	\mathbf{l} &=& \ell_{11}^{-1} \mathbf{a}	\\
	\mathbf{L}_{22} \mathbf{L}_{22}^T &=& \mathbf{A}_{22}  - \mathbf{l} \mathbf{l}^T = \mathbf{A}_{22}  - a_{11}^{-1} \mathbf{a} \mathbf{a}^T.
\end{eqnarray*}
Now $a_{11}>0$ (why?), so $\ell_{11}$ and $\mathbf{l}$ are uniquely determined. $\mathbf{A}_{22} - a_{11}^{-1} \mathbf{a} \mathbf{a}^T$ is positive definite because $\mathbf{A}$ is positive definite (why?). By induction hypothesis, $\mathbf{L}_{22}$ exists and is unique.

* The constructive proof completely specifies the algorithm. 

<img src="http://www.netlib.org/utk/papers/factor/_25826_figure440.gif" width="500" align="center"/>

* Computational cost: 
$$
\frac 12 [2(n-1)^2 + 2(n-2)^2 + \cdots + 2 \cdot 1^2] \approx \frac 13 n^3 \quad \text{flops}
$$ 
plus $n$ square roots. Half the cost of LU decomposition by utilizing symmetry.

* In general Cholesky decomposition is very stable. Failure of the decomposition simply means $\mathbf{A}$ is not positive definite. It is an efficient way to test positive definiteness.  


## Pivoting

* When $\mathbf{A}$ does not have full rank, e.g., $\mathbf{X}^T \mathbf{X}$ with a non-full column rank $\mathbf{X}$, we encounter $a_{kk} = 0$ during the procedure.

* **Symmetric pivoting**. At each stage $k$, we permute both row and column such that $\max_{k \le i \le n} a_{kk}$ becomes the pivot. If we encounter $\max_{k \le i \le n} a_{kk} = 0$, then $\mathbf{A}[k:n,k:n] = \mathbf{0}$ (why?) and the algorithm terminates.

* With symmetric pivoting: 
$$
\mathbf{P} \mathbf{A} \mathbf{P}^T = \mathbf{L} \mathbf{L}^T,
$$
where $\mathbf{P}$ is a permutation matrix and $\mathbf{L} \in \mathbb{R}^{n \times r}$, $r = \text{rank}(\mathbf{A})$.

## Implementation

* LAPACK functions: `?potrf` (without pivoting), `?pstrf` (with pivoting).

* Julia functions: [`cholfact`](https://docs.julialang.org/en/stable/stdlib/linalg/?highlight=cholfact#Base.cholfact), [`cholfact!`](https://docs.julialang.org/en/stable/stdlib/linalg/?highlight=cholfact#Base.cholfact!), [`chol`](https://docs.julialang.org/en/stable/stdlib/linalg/?highlight=cholfact#Base.chol), or call LAPACK wrapper functions [`potrf!`](https://docs.julialang.org/en/stable/stdlib/linalg/?highlight=blas#Base.LinAlg.LAPACK.potrf!) and [`pstrf!`](https://docs.julialang.org/en/stable/stdlib/linalg/?highlight=blas#Base.LinAlg.LAPACK.pstrf!)

In [1]:
A = [4 12 -16; 12 37 -43; -16 -43 98]

3×3 Array{Int64,2}:
   4   12  -16
  12   37  -43
 -16  -43   98

In [2]:
# Cholesky without pivoting
Achol = cholfact(A)

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
[2.0 6.0 -8.0; 0.0 1.0 5.0; 0.0 0.0 3.0]

In [3]:
Achol[:L]

3×3 LowerTriangular{Float64,Array{Float64,2}}:
  2.0   ⋅    ⋅ 
  6.0  1.0   ⋅ 
 -8.0  5.0  3.0

In [4]:
Achol[:U]

3×3 UpperTriangular{Float64,Array{Float64,2}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

In [5]:
b = [1.0; 2.0; 3.0]
A \ b # this does LU decomposition

3-element Array{Float64,1}:
 28.5833 
 -7.66667
  1.33333

In [6]:
Achol \ b # two triangular solves

3-element Array{Float64,1}:
 28.5833 
 -7.66667
  1.33333

In [7]:
det(A) # this actually does LU

36.0

In [8]:
det(Achol) # cheap

36.0

In [15]:
inv(A) # does LU

3×3 Array{Float64,2}:
  49.3611   -13.5556     2.11111 
 -13.5556     3.77778   -0.555556
   2.11111   -0.555556   0.111111

In [18]:
inv(Achol)

3×3 Array{Float64,2}:
  49.3611   -13.5556     2.11111 
 -13.5556     3.77778   -0.555556
   2.11111   -0.555556   0.111111

## Applications

* **No inversion** mentality: Whenever we see matrix inverse, we should think in terms of solving linear equations. If the matrix is positive (semi)definite, use Cholesky decomposition.

* Example: multivariate normal density $\text{MVN}(\mu, \Sigma)$,  $\Sigma$ is p.d., is
$$
	- \frac n2 \ln (2\pi) - \frac 12 \ln \det \Sigma - \frac 12 (\mathbf{y} - \mu)^T \Sigma^{-1} (\mathbf{y} - \mu).
$$

    * Method 1: (a) compute explicit inverse $\Sigma^{-1}$ ($2n^3$ flops), (b) compute quadratic form ($2n^2 + 2n$ flops), (c) compute determinant ($2n^3/3$ flops).
    
    * Method 2: (a) Cholesky decomposition $\Sigma = \mathbf{L} \mathbf{L}^T$ ($n^3/3$ flops), (b) Solve $\mathbf{L} \mathbf{x} = \mathbf{y} - \mu$ by forward substitutions ($n^2$ flops), (c) compute quadratic form $\mathbf{x}^T \mathbf{x}$ ($2n$ flops), and (d) compute determinant from Cholesky factor ($n$ flops).  

    Which method is better?

* Compute Moore-Penrose inverse $\mathbf{A}^+$.

* Cholesky decomposition is **one** approach to solve linear regression.

In [9]:
function logpdf_mvn_1(y::Vector, Σ::Matrix)
    n = length(y)
    - (n//2) * log(2π) - (1//2) * logdet(Σ) - (1//2) * dot(y, inv(Σ) * y)
end

function logpdf_mvn_2(y::Vector, Σ::Matrix)
    n = length(y)
    Σchol = cholfact(Σ)
    - (n//2) * log(2π) - (1//2) * logdet(Σchol) - (1//2) * sumabs2(Σchol[:L] \ y)
end

logpdf_mvn_2 (generic function with 1 method)

In [10]:
using BenchmarkTools, Distributions

srand(280) # seed

n   = 1000
Σ   = randn(n, n); Σ = Σ' * Σ + I # covariance matrix
mvn = MvNormal(Σ) # N(0, Σ)
y   = rand(mvn) # one sample

@show logpdf_mvn_1(y, Σ)
@show logpdf_mvn_2(y, Σ)

logpdf_mvn_1(y,Σ) = -4396.173284372774
logpdf_mvn_2(y,Σ) = -4396.173284372773


-4396.173284372773

In [11]:
@benchmark logpdf_mvn_1(y, Σ)

BenchmarkTools.Trial: 
  memory estimate:  23.41 MiB
  allocs estimate:  21
  --------------
  minimum time:     68.442 ms (3.38% GC)
  median time:      69.719 ms (3.23% GC)
  mean time:        70.055 ms (3.24% GC)
  maximum time:     74.417 ms (2.97% GC)
  --------------
  samples:          72
  evals/sample:     1

In [12]:
@benchmark logpdf_mvn_2(y, Σ)

BenchmarkTools.Trial: 
  memory estimate:  15.27 MiB
  allocs estimate:  17
  --------------
  minimum time:     12.422 ms (0.00% GC)
  median time:      14.647 ms (14.60% GC)
  mean time:        14.243 ms (12.28% GC)
  maximum time:     23.251 ms (9.53% GC)
  --------------
  samples:          350
  evals/sample:     1

## Further reading

* Section 7.7 of [Numerical Analysis for Statisticians](http://ucla.worldcat.org/title/numerical-analysis-for-statisticians/oclc/793808354&referer=brief_results) of Kenneth Lange (2010).

* Section II.5.3 of [Computational Statistics](http://ucla.worldcat.org/title/computational-statistics/oclc/437345409&referer=brief_results) by James Gentle (2010).

* Section 4.2 of [Matrix Computation](http://catalog.library.ucla.edu/vwebv/holdingsInfo?bibId=7122088) by Gene Golub and Charles Van Loan (2013).